In [12]:
import os
import numpy as np

from keras.preprocessing.image import ImageDataGenerator
import keras
import datetime
import sys  
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Activation
from keras.activations import relu, tanh, elu
from keras.optimizers import Adagrad, Adam, Nadam, SGD
from keras.losses import categorical_crossentropy
from keras.layers.normalization import BatchNormalization
from keras.constraints import maxnorm
from keras import optimizers
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K

import tensorflow as tf
from keras.callbacks import ModelCheckpoint

from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions


In [13]:
def preprocess_input(x):
    return tf.keras.applications.resnet50.preprocess_input(x)

In [14]:
data_generator_train = ImageDataGenerator(
    featurewise_center=False,  
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False, 
    rotation_range=30, 
    zoom_range=0.3,
    horizontal_flip=True, 
    vertical_flip=False,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.001,
    channel_shift_range=0.1,
    fill_mode='reflect',
    data_format='channels_last',
   #preprocessing_function=preprocess_input  
)



data_generator_val = ImageDataGenerator(
    featurewise_center=False,  
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False, 
     data_format='channels_last',
  #preprocessing_function=preprocess_input
)

data_generator_test = ImageDataGenerator(
    featurewise_center=False,  
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
     data_format='channels_last',
   #preprocessing_function=preprocess_input
)


train_generator = data_generator_train.flow_from_directory(
    'E:/My_Dataset/Training_data',   
    target_size=(250, 250),
    batch_size=100
)

val_generator = data_generator_val.flow_from_directory(
    'E:/My_Dataset/Validation_data', shuffle=False,
    target_size=(250, 250),
    batch_size=20
)

test_generator = data_generator_test.flow_from_directory(
    'E:/My_Dataset/Testing_data', shuffle=False, target_size = (250,250), batch_size = 20)

Found 39431 images belonging to 6 classes.
Found 4614 images belonging to 6 classes.
Found 2078 images belonging to 6 classes.


In [19]:
classes = 6
weight_decay=1e-5
lr = 1e-3
epochs = 20
decay = lr/epochs

# Constructing the Model 
feature_extractor = keras.applications.resnet50.ResNet50(input_shape=(250, 250, 3), include_top=False, weights='imagenet', pooling = 'avg', classes=classes)
# make sure eno 3rd dimension is 3 ????


#feature_extractor = keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(250, 250, 3), pooling='avg', classes=classes)
#feature_extractor = keras.applications.vgg19.VGG19(include_top=False, weights='imagenet', input_shape=(250, 250, 3), pooling='avg', classes=classes)
# feature_extractor = keras.applications.mobilenet.MobileNet(input_shape=(250, 250, 3), alpha=1.0, depth_multiplier=1, dropout=1e-3, include_top=False, weights='imagenet', pooling='avg', classes=classes)


for layer in feature_extractor.layers[:]:
        layer.trainable = False
        
classifier = feature_extractor.output
classifier = Dropout(0.5)(classifier)
logits1 = Dense(100, activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)) (classifier)
logits2 = Dense(50, activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)) (logits1)
logits3 = Dense(classes, activation='relu', kernel_regularizer=keras.regularizers.l2(weight_decay)) (logits2)
probabilities = Activation('softmax') (logits3)
    
full_model = Model(feature_extractor.input, probabilities)

nadam = Nadam(lr=lr,beta_1=0.9, beta_2 = 0.999, epsilon = 1e-8, schedule_decay=decay)
full_model.compile(loss='categorical_crossentropy', optimizer=nadam, metrics=['accuracy'])

In [ ]:
check_point = ModelCheckpoint("my_best_weights.hdf5", monitor = "val_acc", save_best_only = True, period = 1)
batch_size = 100
history = full_model.fit_generator(  train_generator,
                                steps_per_epoch= 39431 // batch_size,
                                epochs=epochs,
                                validation_data=val_generator,
                                verbose=1,
                                workers=8, 
                                callbacks=[check_point],
                             )

Epoch 1/20
262/394 [==================>...........] - ETA: 5:38 - loss: 1.4968 - acc: 0.4078

C:\Users\Lenovo\Anaconda2\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4718592 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\Lenovo\Anaconda2\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 787528 bytes but only got 5533. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\Lenovo\Anaconda2\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 7281180688 bytes but only got 0. Skipping tag 8
  " Skipping tag %s" % (size, len(data), tag))


273/394 [===================>..........] - ETA: 5:09 - loss: 1.4940 - acc: 0.4092

C:\Users\Lenovo\Anaconda2\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 787528 bytes but only got 5401. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))


394/394 [==============================] - 1206s 3s/step - loss: 1.4726 - acc: 0.4202 - val_loss: 1.4796 - val_acc: 0.4330
Epoch 2/20
394/394 [==============================] - 3642s 9s/step - loss: 1.3760 - acc: 0.4797 - val_loss: 1.3834 - val_acc: 0.4933
Epoch 3/20
394/394 [==============================] - 1567s 4s/step - loss: 1.2824 - acc: 0.5195 - val_loss: 1.2128 - val_acc: 0.5262
Epoch 4/20
213/394 [===============>..............] - ETA: 12:00 - loss: 1.1355 - acc: 0.5659